In [1]:
import os
import numpy as np
import pickle
import pandas as pd
from scipy.stats import zscore
from brainbox.io.one import SessionLoader
from sklearn.preprocessing import StandardScaler
import gc
import concurrent.futures

from functions import merge_licks, resample_common_time, interpolate_nans, low_pass
from one.api import ONE
one = ONE(mode='remote')


In [2]:

""" PARAMETERS """
SAMPLING = {'left': 60,
            'right': 150,
            'body': 30}

In [5]:
""" Load BWM data post-QC """
prefix = '/home/ines/repositories/'
# prefix = '/Users/ineslaranjeira/Documents/Repositories/'
data_path = prefix + '/representation_learning_variability/paper-individuality/'
filename = '1_bwm_qc_07-10-2025'



data_path = prefix + 'representation_learning_variability/Video and wheel/Video QC/'
filename = 'final_lp_qc07-10-2025'


bwm_query = pickle.load(open(data_path+filename, "rb"))


/tmp/ipykernel_16430/431073403.py:13: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  bwm_query = pickle.load(open(data_path+filename, "rb"))


In [6]:
# Loop through animals
sessions = bwm_query['eid'].unique()
data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/'
os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)

    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))

len(sessions_to_process)


218

In [8]:
def process_design_matrix(session):   

    file_path = one.eid2path(session)
    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    """ LOAD VARIABLES """
    sl = SessionLoader(eid=session, one=one)
    sl.load_pose(views=['left', 'right'], tracker='lightningPose')
    sl.load_session_data(trials=True, wheel=True, motion_energy=True)

    # Check if all data is available
    if np.sum(sl.data_info['is_loaded']) >= 4:

        # Poses
        poses = sl.pose
        lc_t = np.asarray(poses['leftCamera']['times'])
        rc_t = np.asarray(poses['rightCamera']['times'])
        # Motion energy
        me = sl.motion_energy
        mel_t = lc_t
        mer_t = rc_t
        motion_energy_l = interpolate_nans(me['leftCamera']['whiskerMotionEnergy'], 'left')
        motion_energy_r = low_pass(interpolate_nans(me['rightCamera']['whiskerMotionEnergy'], 'right'), 
                                          cutoff=30, sf=SAMPLING['right'])
        # Licks
        features = ['tongue_end_l_x', 'tongue_end_l_y', 'tongue_end_r_x', 'tongue_end_r_y']
        lick_t, licks = merge_licks(poses, features, common_fs=150)
        # Paws
        l_paw_x = interpolate_nans(poses['leftCamera']['paw_r_x'], 'left')
        l_paw_y = interpolate_nans(poses['leftCamera']['paw_r_y'], 'left')
        r_paw_x = low_pass(interpolate_nans(poses['rightCamera']['paw_r_x'], 'right'), 
                                          cutoff=30, sf=SAMPLING['right'])
        r_paw_y = low_pass(interpolate_nans(poses['rightCamera']['paw_r_y'], 'right'), 
                                          cutoff=30, sf=SAMPLING['right'])
        l_paw_t = lc_t
        r_paw_t = rc_t
        # Wheel
        wheel = sl.wheel
        wheel_t = np.asarray(wheel['times'], dtype=np.float64)
        wheel_vel = wheel['velocity'].astype(np.float32)

        # Common resampling window
        onset = max(lc_t.min(), rc_t.min(), wheel_t.min(), lick_t.min())
        offset = min(lc_t.max(), rc_t.max(), wheel_t.max(), lick_t.max())
        fs = 60
        ref_t = np.arange(onset, offset, 1 / fs, dtype=np.float64)

        # Restrict to time window
        def restrict(t, x):
            mask = (t >= onset) & (t <= offset)
            return t[mask], x[mask]

        mel_t, motion_energy_l = restrict(mel_t, motion_energy_l)
        mer_t, motion_energy_r = restrict(mer_t, motion_energy_r)
        wheel_t, wheel_vel = restrict(wheel_t, wheel_vel)
        l_paw_t_x, l_paw_x = restrict(l_paw_t, l_paw_x)
        l_paw_t_y, l_paw_y = restrict(l_paw_t, l_paw_y)
        r_paw_t_x, r_paw_x = restrict(r_paw_t, r_paw_x)
        r_paw_t_y, r_paw_y = restrict(r_paw_t, r_paw_y)
        lick_t, licks = restrict(lick_t, licks)

        # Resample
        mel_down, rt = resample_common_time(ref_t, mel_t, motion_energy_l, kind='linear')
        mer_down, _ = resample_common_time(ref_t, mer_t, motion_energy_r, kind='linear')
        wh_down, _ = resample_common_time(ref_t, wheel_t, wheel_vel, kind='linear')
        lk_down, _ = resample_common_time(ref_t, lick_t, licks, kind='nearest')
        lpx_down, _ = resample_common_time(ref_t, l_paw_t_x, l_paw_x, kind='linear')
        lpy_down, _ = resample_common_time(ref_t, l_paw_t_y, l_paw_y, kind='linear')
        rpx_down, _ = resample_common_time(ref_t, r_paw_t_x, r_paw_x, kind='linear')
        rpy_down, _ = resample_common_time(ref_t, r_paw_t_y, r_paw_y, kind='linear')

        # Create design matrix
        design_matrix = pd.DataFrame({
            'Bin': rt,
            'Lick count': lk_down.astype(np.int8),
            'avg_wheel_vel': wh_down,
            'whisker_me': mel_down,  # zscore(mel_down, nan_policy='omit'),
            'avg_whisker_me': np.nanmean([mel_down, mer_down], axis=0),  # zscore(np.nanmean([mel_down, mer_down], axis=0), nan_policy='omit'),
            'l_paw_x': lpx_down,
            'l_paw_y': lpy_down,
            'r_paw_x': rpx_down,
            'r_paw_y': rpy_down,
        })

        # """ LOAD TRIAL DATA """
        session_trials = sl.trials
        session_start = list(session_trials['goCueTrigger_times'])[0]
        design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]

        """ SAVE DATA """       
        # Save unnormalized design matrix
        filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
        design_matrix.to_parquet(filename, compression='gzip')  

        # Save trials
        filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
        session_trials.to_parquet(filename, compression='gzip')  
        
        del design_matrix, session_trials, sl
        gc.collect()

    else:
        print('Data missing for session '+session)  


def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

In [10]:
for s, session in enumerate(sessions_to_process[:20]):
    process_design_matrix(session)

2025-09-12 11:58:47 INFO     one.py:1475 Loading trials data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-09-12 11:58:47 INFO     one.py:1475 Loading wheel data
2025-09-12 11:58:48 INFO     one.py:1475 Loading motion_energy data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-29"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-31"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-09-12 11:58:48 INFO     one.py:1475 Loading pupil data


...

2025-09-12 12:04:06 INFO     one.py:1475 Loading wheel data


2025-09-12 12:04:06 INFO     one.py:1475 Loading motion_energy data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-29"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-25/001/alf/#2025-05-29#/leftCamera.ROIMotionEnergy.npy: 100%|██████████| 1.93M/1.93M [00:01<00:00, 1.57MB/s]
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-31"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-25/001/alf/#2025-05-31#/rightCamera.ROIMotionEnergy.npy: 100%|██████████| 4.83M/4.83M [00:01<00:00, 2.86MB/s]
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-06-01"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.AL

2025-09-12 12:04:14 INFO     one.py:1475 Loading pupil data



/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "2025-06-18", ""
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-25/001/alf/#2025-06-18#/_ibl_leftCamera.features.pqt: 100%|██████████| 4.44M/4.44M [00:01<00:00, 2.26MB/s]
